<a href="https://colab.research.google.com/github/Alirezanltv/compression_based_on_region_of_interests/blob/main/Progressiveness_video_cif_frames_HEVC_6_jan_(Transmitter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install requirements

In [ ]:
!pip install ffmpeg-python
!pip install yolov5

# drive should be mounted

In [ ]:
from google.colab import drive
drive.mount("/drive", force_remount=True)

!mv "/drive/My Drive/YOLO_weights/last_AFO4.pt" "/content"

Mounted at /drive


# import libraries

In [ ]:
# imports

from google.colab.patches import cv2_imshow
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import cv2
from yolov5 import YOLOv5

# set object detection model with its weights and determine the device

In [ ]:
# set model params
model_path = "Yolov5/weights/yolov5l.pt" # it automatically downloads yolov5l model to given path 
device = "cpu" # or "cpu" or "cuda"

# init yolov5 model
yolov5 = YOLOv5(model_path, device)

  0%|          | 0.00/89.2M [00:00<?, ?B/s]

# read the video

In [ ]:
# Opens the Video file
cap= cv2.VideoCapture('directory OR camera ip')

# progressiveness module

progressiveness images are saved in " result_cif_p"

In [ ]:
os.mkdir('/content/result_cif_p')
count=0
while(cap.isOpened()):
    ret, frame = cap.read()
   
    if ret == False:
        break
    if count==0 or count%9==0:
      frame = cv2.resize(frame ,(708,576))
      predicted = yolov5.predict(frame)
      # predicted = yolov5.predict(frame, augment=True)
      bb_tensors =[]
      bb_tensors.append(predicted.xyxy[0])
      # print(bb_tensors)
      num_boxes =[]
      num_boxes = len(bb_tensors)
      # print(num_boxes)

      rec =[]

      for j in range(0,num_boxes):
        rec.append(bb_tensors[j][0:4].cpu().numpy())
      rec = np.array(rec)

      # sudo fibonacci
      a =0
      b=[0]
      for t in [num_boxes]:
        a+=t
        b.append(a)
      # print(b)

      REC_matrix =[]
      for i in range(0,len(b)) :
        if i==len(b)-1:
          break
        REC_matrix.append(rec[b[i]:b[i+1]])
      REC_matrix = np.array(REC_matrix)
      print(REC_matrix)
      # ////////////////////////////////////////////////////////////////////////////////////////

      # ///////////////////////////////////////////////////////////////////////////////
      # CIF images should be making
      img = cv2.resize(frame , (frame.shape[1]//2 , frame.shape[0]//2))
      filter_mean =np.zeros((img.shape[0] , img.shape[1] , 3))
      block_size =13
    
      for i in range(0,img.shape[0]-block_size):
        for j in range(0, img.shape[1]-block_size):
          for k in range(0,3):
            filter_mean[i*block_size:block_size*(i+1) , j*block_size: block_size*(j+1) , k]=np.sum(img[i*block_size:block_size*(i+1) , j*block_size: block_size*(j+1),k]) /(block_size**2)
      
      REC_matrix_cif = REC_matrix//2
      REC_matrix_cif = REC_matrix_cif[0][0]
      
      mask = np.zeros((img.shape[0] , img.shape[1] , 3))


      for k in range(0,len(REC_matrix_cif)):

        mask[int(REC_matrix_cif[k][1]):int(REC_matrix_cif[k][3]) ,int(REC_matrix_cif[k][0]):int(REC_matrix_cif[k][2])]=1
      
      Mbb = np.zeros((img.shape[0] , img.shape[1] , 3))

      for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
          Mbb[i][j] = mask[i][j]*img[i][j]


      for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
          for k in range(0,3):
            if Mbb[i][j][k]==0:
              Mbb[i][j][k] = Mbb[i][j][k]+filter_mean[i][j][k]

    cv2_imshow(Mbb)
    cv2.imwrite("/content/result_cif_p/"+str(count)+".jpg" , Mbb)
    count+=1

# Making  video out of frames

In [ ]:

import os
def save():
    os.system("ffmpeg -r 3 -i /content/result_cif_p/%01d.jpg -vcodec mpeg4 -y compressed_prog.mp4")
save()

In [ ]:
# len(os.listdir("/content/result_cif_p"))

257

# the name of your video would be " compressed_prog"

it should be compressed with h265 codec and the output file video name is "cif_hvc"

In [ ]:
# finally it works
!ffmpeg -i /content/compressed_prog.mp4 -vcodec libx265 -crf 36 -preset medium -c:a aac  -x265-params bframes=3    -pix_fmt yuv420p  cif_hvc.mp4

# without progressiveness

images will be save in this folder called "frames"

In [ ]:
# Opens the Video file
cap= cv2.VideoCapture('/content/bombers_ships_2.mp4')

In [ ]:
os.mkdir('/content/frames')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
   
    if ret == False:
        break
    if i==0 or i%10 ==0:
      frame = cv2.resize(frame ,(356,288))
      cv2.imwrite('/content/frames/'+str(i//10)+'.jpg',frame)
    i+=1


# Making  video out of frames
print(len(os.listdir("/content/frames")))




84


# making video out of frames

In [ ]:
import os
def save():
    os.system("ffmpeg -r 3 -i /content/frames/%01d.jpg -vcodec mpeg4 -y cif_bombers_10.mp4")
save()

# it should be compressed with h265 codec.
your output video name is cif_hvc_10

In [ ]:
# finally it works
!ffmpeg -i /content/cif_bombers_10.mp4 -vcodec libx265 -crf 36 -preset medium -c:a aac  -x265-params bframes=3    -pix_fmt yuv420p  /content/cif_hvc_10.mp4